# 用户点击日志预处理

将点击日志处理为文本序列

In [1]:
import numpy as np
import pandas as pd

In [2]:
import gc

## 载入用户点击日志

In [3]:
df_user_click_log_train = pd.read_csv("./data/train_preliminary/click_log.csv")
df_user_click_log_test = pd.read_csv("./data/test/click_log.csv")

In [ ]:
df_user_click_log_train.shape,df_user_click_log_test.shape

In [5]:
df_user_click_log_train['click_times'].describe(percentiles=[0.05,0.25,0.5,0.75,0.95,0.99])

count    3.008277e+07
mean     1.063127e+00
std      2.982831e-01
min      1.000000e+00
5%       1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
95%      2.000000e+00
99%      2.000000e+00
max      1.520000e+02
Name: click_times, dtype: float64

### 广告信息

In [6]:
df_ad_info_train = pd.read_csv("./data/train_preliminary/ad.csv")
df_ad_info_test = pd.read_csv("./data/test/ad.csv")

In [11]:
df_ad_info_train.shape,df_ad_info_test.shape

((2481135, 6), (2618159, 6))

In [12]:
df_ad_info_train['industry'].min(),df_ad_info_train['industry'].max()

('1', '\\N')

In [9]:
df_ad_info_test['industry'].min(),df_ad_info_test['industry'].max()

('1', '\\N')

In [10]:
df_ad_info_train['product_id'].min(),df_ad_info_train['product_id'].max()

('1', '\\N')

In [127]:
df_ad_info_train.dtypes

creative_id          int64
ad_id                int64
product_id          object
product_category     int64
advertiser_id        int64
industry            object
dtype: object

In [131]:
df_ad_info_test.dtypes

creative_id          int64
ad_id                int64
product_id          object
product_category     int64
advertiser_id        int64
industry            object
dtype: object

### 标签信息

In [7]:
df_user_label = pd.read_csv("./data/train_preliminary/user.csv")

In [8]:
df_user_label.shape

(900000, 3)

## 合并训练集和测试集点击日志

In [9]:
df_user_features_train = pd.merge(df_user_click_log_train,df_ad_info_train,on='creative_id')

In [10]:
df_user_features_test = pd.merge(df_user_click_log_test,df_ad_info_test,on='creative_id')

In [11]:
df_user_features = pd.concat((df_user_features_train,df_user_features_test))

In [18]:
df_user_features.shape

(63668283, 9)

In [144]:
df_user_features.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,15,320815,567330,1,504423,30673,3,32638,319
2,11,355089,567330,1,504423,30673,3,32638,319
3,9,363442,567330,1,504423,30673,3,32638,319
4,14,370513,567330,1,504423,30673,3,32638,319


In [145]:
df_user_features['industry'].min(),df_user_features['industry'].max()

('1', '\\N')

In [12]:
del df_user_features_train
del df_user_features_test

In [13]:
gc.collect()

0

In [14]:
for x in df_user_features.columns:
    print("%s:%s" % (x,str(df_user_features[x].min())))

time:1
user_id:1
creative_id:1
click_times:1
ad_id:1
product_id:1
product_category:1
advertiser_id:1
industry:1


In [15]:
df_user_features.memory_usage().sum()/1024**2

4857.504501342773

## 替换缺失值

In [16]:
df_user_features.replace('\\N',0,inplace=True)

## 压缩数据

In [17]:
df_user_features.memory_usage().sum()/1024**2

4857.504501342773

In [18]:
np.iinfo(np.int32).max

2147483647

In [19]:
for col in df_user_features.columns:
    df_user_features[col] = df_user_features[col].astype(np.int32)

In [20]:
df_user_features.memory_usage().sum()/1024**2

2671.6274757385254

In [29]:
for x in df_user_features.columns:
    print("%s:%s" % (x,str(df_user_features[x].max())))

time:91
user_id:4000000
creative_id:4445720
click_times:185
ad_id:3812202
product_id:44314
product_category:18
advertiser_id:62965
industry:335


In [21]:
df_user_features.dtypes

time                int32
user_id             int32
creative_id         int32
click_times         int32
ad_id               int32
product_id          int32
product_category    int32
advertiser_id       int32
industry            int32
dtype: object

## 排序

In [22]:
df_user_features.sort_values(by=['user_id','time'],inplace=True,ascending=[True,True])

In [34]:
df_user_features.head(20)

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
18768377,76,1,3592928,2,3093461,1261,2,10988,6
8869317,73,1,2456345,1,2116146,64,2,188,245
8869318,64,1,2456345,1,2116146,64,2,188,245
15516145,64,1,3068256,1,2638858,1454,2,23575,238
15704991,60,1,2087846,1,1803758,0,18,44865,64
10203710,52,1,90171,1,82421,0,5,367,217
13223979,46,1,1940159,1,1678901,0,5,17189,73
5298558,43,1,71691,1,66210,0,18,14681,326
9498187,40,1,122032,1,109959,1334,2,11411,0
22502265,39,1,1683713,1,1458878,0,5,14668,326


In [35]:
df_user_features['click_times'].describe(percentiles=[0.05,0.25,0.5,0.75,0.95,0.99,0.999,0.9999])

count     6.366828e+07
mean      1.064746e+00
std       3.300665e-01
min       1.000000e+00
5%        1.000000e+00
25%       1.000000e+00
50%       1.000000e+00
75%       1.000000e+00
95%       2.000000e+00
99%       2.000000e+00
99.9%     4.000000e+00
99.99%    8.000000e+00
max       1.850000e+02
Name: click_times, dtype: float64

In [177]:
df_user_features['click_times_smooth'] = df_user_features['click_times'].apply(lambda x:min(x,10))

In [178]:
df_user_features['click_times_smooth'].describe(percentiles=[0.05,0.25,0.5,0.75,0.95,0.99,0.999,0.9999])

count     6.366828e+07
mean      1.064385e+00
std       3.012934e-01
min       1.000000e+00
5%        1.000000e+00
25%       1.000000e+00
50%       1.000000e+00
75%       1.000000e+00
95%       2.000000e+00
99%       2.000000e+00
99.9%     4.000000e+00
99.99%    8.000000e+00
max       1.000000e+01
Name: click_times_smooth, dtype: float64

In [179]:
df_user_features.isnull().sum()

time                  0
user_id               0
creative_id           0
click_times           0
ad_id                 0
product_id            0
product_category      0
advertiser_id         0
industry              0
click_times_smooth    0
dtype: int64

## 合并用户点击序列

In [36]:
df_user_features.groupby("user_id").head(20)

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
18768377,76,1,3592928,2,3093461,1261,2,10988,6
8869317,73,1,2456345,1,2116146,64,2,188,245
8869318,64,1,2456345,1,2116146,64,2,188,245
15516145,64,1,3068256,1,2638858,1454,2,23575,238
15704991,60,1,2087846,1,1803758,0,18,44865,64
...,...,...,...,...,...,...,...,...,...
30802063,60,4000000,2769944,1,2381896,0,18,45380,133
12161667,59,4000000,2490312,1,2144917,0,18,41654,26
29153627,59,4000000,2687277,1,2312216,2046,2,14155,247
24319295,58,4000000,2500260,1,2153299,0,18,24747,54


In [38]:
def extract_user_creative_id(dfData,min_word_num=0):
    '''
    提取用户文本内容
    Parameters
    ----------
    dfData：pandas.DataFrame，需包含列名['user_id','creative_id','ad_id','time','click_times','product_id','product_category','advertiser_id','industry']
    min_word_num：最小词数要求，默认为0
    Returns
    ----------
    user_list：list,用户各个id的分词序列
    '''
    user_list = []
    count=0
    exclude_num=0#不满足最小词数要求的用户数
    for uid,group in dfData.groupby("user_id"):
        userinfo=[]
        count+=1
        click_times_list = group["click_times"].values
        click_times_len = click_times_list.shape[0]
        userinfo.append(uid)
        for id_col in ['time','creative_id','click_times','ad_id','product_id','product_category','advertiser_id','industry']:
            id_list = group[id_col].values
            segtitle=''
            for i in range(0,click_times_len):
                segtitle+=(str(id_list[i])+" ")#*click_times_list[i]
            #segtitle = " ".join(seglist)
            segtitle = segtitle.strip()
            segtitle_len = len(segtitle.split(" "))
            if segtitle!="":
                if segtitle_len>=min_word_num:  
                    userinfo.append(segtitle)
                else:
                    exclude_num+=1
                    if(exclude_num<=3):
                        print("exclude_uid:",uid)
        if count<=3:
            print("uid:",uid)
            print("userinfo:",userinfo)
        if count%100000==0:
            print(count)
        user_list.append(userinfo)
    print("exclude_num:",exclude_num)
    print("处理记录总数量：",count)
    return user_list

In [39]:
user_ad_list = extract_user_creative_id(df_user_features)

uid: 1
userinfo: [1, '76 73 64 64 60 52 46 43 40 39 20 20 20', '3592928 2456345 2456345 3068256 2087846 90171 1940159 71691 122032 1683713 877468 209778 821396', '2 1 1 1 1 1 1 1 1 1 1 1 1', '3093461 2116146 2116146 2638858 1803758 82421 1678901 66210 109959 1458878 773445 188507 724607', '1261 64 64 1454 0 0 0 0 1334 0 0 136 0', '2 2 2 2 18 5 5 18 2 5 5 2 5', '10988 188 188 23575 44865 367 17189 14681 11411 14668 29455 9702 7293', '6 245 245 238 64 217 73 326 0 326 106 6 326']
uid: 2
userinfo: [2, '90 89 88 87 87 85 78 77 77 77 73 72 68 67 60 60 60 59 55 53 53 53 53 51 49 49 49 49 48 44 44 42 42 42 41 39 38 38 28 28 28 17 14 11 10', '625944 4194247 513298 22013 15558 2270719 3574990 1416518 3765766 3766271 2995416 3443654 3246898 738378 2785305 2108551 2862407 2624965 2496722 647980 2143574 2489057 2552139 2369339 2085566 157180 396652 1696925 2085566 1145367 96192 1252062 1145367 1662244 1074235 769749 1696925 1657530 13069 1266180 441462 609050 39714 155822 63441', '1 1 1 1 2 1 1 1 

In [40]:
pd_user_ad_list = pd.DataFrame(user_ad_list,columns=['user_id','time','creative_id','click_times','ad_id','product_id','product_category','advertiser_id','industry'])

导出预处理结果

In [41]:
pd_user_ad_list.to_csv("./data/pd_user_ad_id_seg_all_click_time.csv",index=False)

In [278]:
pd_user_ad_list.head()

,user_id,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,1,20 20 20 39 40 43 46 52 60 64 64 73 76,877468 209778 821396 1683713 122032 71691 1940...,1 1 1 1 1 1 1 1 1 1 1 1 2,773445 188507 724607 1458878 109959 66210 1678...,0 136 0 0 1334 0 0 0 0 64 1454 64 1261,5 2 5 5 2 18 5 5 18 2 2 2 2,29455 9702 7293 14668 11411 14681 17189 367 44...,106 6 326 326 0 326 73 217 64 245 238 245 6
1,2,10 11 14 17 28 28 28 38 38 39 41 42 42 42 44 4...,63441 155822 39714 609050 13069 1266180 441462...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,58788 139702 38066 541125 14495 1107111 392680...,87 80 129 129 1400 0 87 0 1261 0 111 129 0 0 0...,2 2 2 2 2 18 2 18 2 18 2 2 18 18 18 18 2 2 2 2...,22885 10686 18562 25932 768 34505 22885 26006 ...,318 238 6 6 317 47 318 47 6 47 242 6 47 6 47 4...
2,3,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...,661347 808612 593522 825434 710859 726940 3920...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,586668 713448 527601 728308 629278 643108 3502...,36256 40905 1674 35985 1674 0 0 1031 1786 2258...,17 17 2 17 2 18 18 2 2 2 18 12 2 2 18 18 8 4 2...,32974 9877 17018 14186 18492 9058 8371 2336 39...,0 0 322 0 322 6 54 6 322 322 205 302 322 322 2...
3,4,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...,39588 589886 574787 1892854 1230094 31070 2264...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,37966 524312 511235 1638619 1076286 30773 1953...,1862 0 2625 38743 39904 39422 37758 41265 0 15...,2 18 2 17 17 17 4 17 18 2 2 2 2 2 2 2 2 2 5 2 ...,19451 7976 13084 12130 10172 13299 811 23664 1...,238 25 248 0 0 0 0 0 88 319 6 238 319 238 319 ...
4,5,3 13 14 15 20 21 24 25 27 28 29 30 32 32 35 35...,296145 350759 24333 43235 852327 1054434 12964...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,265971 314795 24966 41148 751113 925792 113378...,0 0 87 136 0 136 0 0 0 1064 0 2620 0 0 0 136 2...,5 8 2 2 18 2 5 18 18 2 18 2 5 5 18 2 2 18 18 5...,11882 992 22885 9706 38760 2862 17745 31552 22...,297 0 318 6 322 6 288 322 319 238 322 322 203 ...


观察数据分布

In [ ]:
#用户点击id个数
pd_user_ad_list['word_count'] = pd_user_ad_list['click_times'].apply(lambda x:len(x.split(" ")))

In [3]:
#用户单日最大点击次数
def max_day_click_times(item):
    click_list = item.split(" ")
    max_time=1
    for click_time in click_list:
        max_time=max(int(click_time),max_time)
    return max_time

In [206]:
pd_user_ad_list['max_day_click_times'] = pd_user_ad_list['click_times'].apply(max_day_click_times)

In [212]:
pd_user_ad_list['max_day_click_times'].describe(percentiles=[0.05,0.25,0.5,0.75,0.95,0.99,0.999,0.9999])

count     1.900000e+06
mean      1.797689e+00
std       9.286576e-01
min       1.000000e+00
5%        1.000000e+00
25%       1.000000e+00
50%       2.000000e+00
75%       2.000000e+00
95%       3.000000e+00
99%       5.000000e+00
99.9%     9.000000e+00
99.99%    1.600000e+01
max       1.850000e+02
Name: max_day_click_times, dtype: float64

In [213]:
pd_user_ad_list.head()

,user_id,creative_id,click_times,product_id,product_category,advertiser_id,industry,word_count,max_day_click_times
0,1,877468 209778 821396 1683713 122032 71691 1940...,1 1 1 1 1 1 1 1 1 1 1 1 2,0 136 0 0 1334 0 0 0 0 64 1454 64 1261,5 2 5 5 2 18 5 5 18 2 2 2 2,29455 9702 7293 14668 11411 14681 17189 367 44...,106 6 326 326 0 326 73 217 64 245 238 245 6,13,2
1,2,63441 155822 39714 609050 13069 1266180 441462...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,87 80 129 129 1400 0 87 0 1261 0 111 129 0 0 0...,2 2 2 2 2 18 2 18 2 18 2 2 18 18 18 18 2 2 2 2...,22885 10686 18562 25932 768 34505 22885 26006 ...,318 238 6 6 317 47 318 47 6 47 242 6 47 6 47 4...,45,2
2,3,661347 808612 593522 825434 710859 726940 3920...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,36256 40905 1674 35985 1674 0 0 1031 1786 2258...,17 17 2 17 2 18 18 2 2 2 18 12 2 2 18 18 8 4 2...,32974 9877 17018 14186 18492 9058 8371 2336 39...,0 0 322 0 322 6 54 6 322 322 205 302 322 322 2...,30,1
3,4,39588 589886 574787 1892854 1230094 31070 2264...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,1862 0 2625 38743 39904 39422 37758 41265 0 15...,2 18 2 17 17 17 4 17 18 2 2 2 2 2 2 2 2 2 5 2 ...,19451 7976 13084 12130 10172 13299 811 23664 1...,238 25 248 0 0 0 0 0 88 319 6 238 319 238 319 ...,29,1
4,5,296145 350759 24333 43235 852327 1054434 12964...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,0 0 87 136 0 136 0 0 0 1064 0 2620 0 0 0 136 2...,5 8 2 2 18 2 5 18 18 2 18 2 5 5 18 2 2 18 18 5...,11882 992 22885 9706 38760 2862 17745 31552 22...,297 0 318 6 322 6 288 322 319 238 322 322 203 ...,33,2


In [204]:
pd_user_ad_list['word_count'].describe(percentiles=[0.05,0.25,0.5,0.75,0.95,0.99])

count    1.900000e+06
mean     3.350962e+01
std      9.479383e+01
min      2.000000e+00
5%       1.000000e+01
25%      1.500000e+01
50%      2.400000e+01
75%      4.000000e+01
95%      8.900000e+01
99%      1.570000e+02
max      1.139740e+05
Name: word_count, dtype: float64